In [1]:
%pip install scikit-learn xgboost catboost

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor 
from xgboost import XGBRegressor  # Assuming XGBRegressor is from xgboost
from catboost import CatBoostRegressor  # Assuming CatBoostRegressor is from catboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('P:/ML_PROJECTS/movies/data/Rotten_Tomatoes_Movies3.csv')

In [5]:
data.sample(2)

,movie_title,movie_info,critics_consensus,rating,genre,directors,writers,cast,in_theaters_date,on_streaming_date,runtime_in_minutes,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_rating
11727,Reprisal,"Jacob (Frank Grillo), a bank manager haunted b...",NaN,R,"Action & Adventure, Mystery & Suspense",Brian A. Miller,Bryce Hammons,"Bruce Willis, Frank Grillo, Johnathon Schaech,...",8/31/2018,10/16/2018,89.0,Lionsgate,Rotten,0,12,17.0
3926,Bugsy,Bugsy is a character study of mobster Benjamin...,"Stylishly scattered, Bugsy offers cinematic ho...",R,Drama,Barry Levinson,James Toback,"Warren Beatty, Annette Bening, Harvey Keitel, ...",12/13/1991,12/15/1998,136.0,Sony Pictures Home Entertainment,Certified Fresh,85,59,69.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16638 entries, 0 to 16637
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_title         16638 non-null  object 
 1   movie_info          16614 non-null  object 
 2   critics_consensus   8309 non-null   object 
 3   rating              16638 non-null  object 
 4   genre               16621 non-null  object 
 5   directors           16524 non-null  object 
 6   writers             15289 non-null  object 
 7   cast                16354 non-null  object 
 8   in_theaters_date    15823 non-null  object 
 9   on_streaming_date   16636 non-null  object 
 10  runtime_in_minutes  16483 non-null  float64
 11  studio_name         16222 non-null  object 
 12  tomatometer_status  16638 non-null  object 
 13  tomatometer_rating  16638 non-null  int64  
 14  tomatometer_count   16638 non-null  int64  
 15  audience_rating     16386 non-null  float64
dtypes: f

In [7]:
data.describe()

,runtime_in_minutes,tomatometer_rating,tomatometer_count,audience_rating
count,16483.000000,16638.000000,16638.000000,16386.000000
mean,102.391494,60.466522,56.607104,60.470829
std,25.028011,28.587230,66.383800,20.462368
min,1.000000,0.000000,5.000000,0.000000
25%,90.000000,38.000000,12.000000,45.000000
50%,99.000000,66.000000,28.000000,62.000000
75%,111.000000,86.000000,76.000000,77.000000
max,2000.000000,100.000000,497.000000,100.000000


In [8]:
data.isnull().sum()

movie_title              0
movie_info              24
critics_consensus     8329
rating                   0
genre                   17
directors              114
writers               1349
cast                   284
in_theaters_date       815
on_streaming_date        2
runtime_in_minutes     155
studio_name            416
tomatometer_status       0
tomatometer_rating       0
tomatometer_count        0
audience_rating        252
dtype: int64

In [9]:
null_percentage = (data.isnull().sum() / len(data)) * 100
null_percentage

movie_title            0.000000
movie_info             0.144248
critics_consensus     50.060103
rating                 0.000000
genre                  0.102176
directors              0.685179
writers                8.107946
cast                   1.706936
in_theaters_date       4.898425
on_streaming_date      0.012021
runtime_in_minutes     0.931602
studio_name            2.500301
tomatometer_status     0.000000
tomatometer_rating     0.000000
tomatometer_count      0.000000
audience_rating        1.514605
dtype: float64

In [10]:
df = data[['genre', 'runtime_in_minutes',
                       'tomatometer_rating', 'tomatometer_count', 'audience_rating']]

In [11]:
df.head()

,genre,runtime_in_minutes,tomatometer_rating,tomatometer_count,audience_rating
0,"Action & Adventure, Comedy, Drama, Science Fic...",83.0,49,144,53.0
1,Comedy,90.0,86,140,64.0
2,"Comedy, Romance",118.0,68,22,53.0
3,"Classics, Drama",95.0,100,51,97.0
4,"Action & Adventure, Drama, Kids & Family",127.0,89,27,74.0


In [12]:
df.isnull().sum()

genre                  17
runtime_in_minutes    155
tomatometer_rating      0
tomatometer_count       0
audience_rating       252
dtype: int64

In [13]:
df = df.dropna(subset=['genre', 'runtime_in_minutes',
                       'tomatometer_rating', 'tomatometer_count', 'audience_rating'])

In [14]:
df.isnull().sum()

genre                 0
runtime_in_minutes    0
tomatometer_rating    0
tomatometer_count     0
audience_rating       0
dtype: int64

In [15]:
df.head()

,genre,runtime_in_minutes,tomatometer_rating,tomatometer_count,audience_rating
0,"Action & Adventure, Comedy, Drama, Science Fic...",83.0,49,144,53.0
1,Comedy,90.0,86,140,64.0
2,"Comedy, Romance",118.0,68,22,53.0
3,"Classics, Drama",95.0,100,51,97.0
4,"Action & Adventure, Drama, Kids & Family",127.0,89,27,74.0


Example pipeline:

Preprocess text (TF-IDF) → Combine with numerical data → Train model (e.g., XGBoost, Ridge Regression).

In [17]:
# Assuming 'movie_info' is the text column and 'audience_rating' is the target
X_text = df[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1) # This line is modified
y = df['audience_rating']

# Numerical features
X_numerical = df[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]


# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')  # Adjust max_features as needed
X_tfidf = vectorizer.fit_transform(X_text) # Pass the combined text data to fit_transform

# Combine text and numerical features
X_combined = np.hstack((X_tfidf.toarray(), X_numerical))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train Ridge Regression model
model = Ridge()  # Adjust alpha as needed
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("R2 Score:", r2)
print("\n")

Mean Squared Error: 228.8175765668809
R2 Score: 0.43652094134459296




In [91]:
models = {
                "Ridge Regression": Ridge(),
                "Random Forest": RandomForestRegressor(),
                "Decision Tree": DecisionTreeRegressor(),
                "Gradient Boosting": GradientBoostingRegressor(),
                "Linear Regression": LinearRegression(),
                "XGBRegressor": XGBRegressor(),
                "CatBoosting Regressor": CatBoostRegressor(verbose=False),
                "AdaBoost Regressor": AdaBoostRegressor(),
            }

for i,j in models.items():
        model = models[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Model: {i}")
        print("Mean Squared Error:", mse)
        print("R2 Score:", r2)
        print("\n")

Model: Ridge Regression
Mean Squared Error: 228.8175765668809
R2 Score: 0.43652094134459296


Model: Random Forest
Mean Squared Error: 212.85484180169902
R2 Score: 0.4758302762916904


Model: Decision Tree
Mean Squared Error: 396.356788221195
R2 Score: 0.02394408126562786


Model: Gradient Boosting
Mean Squared Error: 193.99159525461062
R2 Score: 0.5222823214842565


Model: Linear Regression
Mean Squared Error: 228.992217976092
R2 Score: 0.43609087483335285


Model: XGBRegressor
Mean Squared Error: 204.15062588244476
R2 Score: 0.4972650081252443


Model: CatBoosting Regressor
Mean Squared Error: 193.69782068865362
R2 Score: 0.5230057616078957


Model: AdaBoost Regressor
Mean Squared Error: 229.15875410848471
R2 Score: 0.43568076812511525




In [27]:
model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("R2 Score:", r2)
print("\n")

Mean Squared Error: 193.69782068865362
R2 Score: 0.5230057616078957




In [28]:
model = joblib.dump(model, 'model.pkl')
vectorizer = joblib.dump(vectorizer, 'vectorizer.pkl')
scaler = joblib.dump(scaler, 'scaler.pkl')

In [24]:
new_movie_data = pd.DataFrame({
    'genre': ['Comedy','Action'],  # Example genre
    'runtime_in_minutes': [12,23], # Example runtime
    'tomatometer_rating': [90,45], # Example tomatometer rating
    'tomatometer_count': [200,100] # Example tomatometer count
})

new_movie_data.head()

,genre,runtime_in_minutes,tomatometer_rating,tomatometer_count
0,Comedy,12,90,200
1,Action,23,45,100


In [25]:
model = joblib.load('P:/ML_PROJECTS/movies/notebook/model.pkl') 
vectorizer = joblib.load('P:/ML_PROJECTS/movies/notebook/vectorizer.pkl') # Replace 'your_vectorizer.joblib' with the actual file path
scaler = joblib.load('P:/ML_PROJECTS/movies/notebook/scaler.pkl')

In [26]:
# Preprocess the new movie data in the same way as the training data
X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
X_tfidf_new = vectorizer.transform(X_text_new)  # Use the same vectorizer
X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
X_combined_new = scaler.transform(X_combined_new) # Use the same scaler

model.predict(X_combined_new).round(1)

array([85. , 56.2])

In [ ]:
# prompt: create streamlit code for above
%pip install streamlit
# !streamlit run app.py & npx localtunnel --port 8501
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import joblib

# Load the trained model and vectorizer (replace with your actual paths)
model = joblib.load('model.pkl') # Replace 'your_model.joblib' with the actual file path
vectorizer = joblib.load('vectorizer.pkl') # Replace 'your_vectorizer.joblib' with the actual file path
scaler = joblib.load('scaler.pkl') #Replace with actual path

st.title('Movie Audience Rating Prediction')

# Input fields for movie features
genre = st.text_input('Genre', 'Comedy') # Added default value
runtime = st.number_input('Runtime (minutes)', min_value=0, value=240)
tomatometer_rating = st.number_input('Tomatometer Rating', min_value=0, max_value=100, value=90)
tomatometer_count = st.number_input('Tomatometer Count', min_value=0, value=200)


if st.button('Predict'):
    # Create a DataFrame from the input features
    new_movie_data = pd.DataFrame({
        'genre': [genre],
        'runtime_in_minutes': [runtime],
        'tomatometer_rating': [tomatometer_rating],
        'tomatometer_count': [tomatometer_count]
    })

    # Preprocess the new movie data
    X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    X_tfidf_new = vectorizer.transform(X_text_new)
    X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
    X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
    X_combined_new = scaler.transform(X_combined_new)

    # Make prediction
    prediction = model.predict(X_combined_new).round(1)[0]

# prompt: create streamlit code for above
%pip install streamlit
# !streamlit run app.py & npx localtunnel --port 8501
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import joblib

# Load the trained model and vectorizer (replace with your actual paths)
model = joblib.load('model.pkl') # Replace 'your_model.joblib' with the actual file path
vectorizer = joblib.load('vectorizer.pkl') # Replace 'your_vectorizer.joblib' with the actual file path
scaler = joblib.load('scaler.pkl') #Replace with actual path

st.title('Movie Audience Rating Prediction')

# Input fields for movie features
genre = st.text_input('Genre', 'Comedy') # Added default value
runtime = st.number_input('Runtime (minutes)', min_value=0, value=240)
tomatometer_rating = st.number_input('Tomatometer Rating', min_value=0, max_value=100, value=90)
tomatometer_count = st.number_input('Tomatometer Count', min_value=0, value=200)


if st.button('Predict'):
    # Create a DataFrame from the input features
    new_movie_data = pd.DataFrame({
        'genre': [genre],
        'runtime_in_minutes': [runtime],
        'tomatometer_rating': [tomatometer_rating],
        'tomatometer_count': [tomatometer_count]
    })

    # Preprocess the new movie data
    X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    X_tfidf_new = vectorizer.transform(X_text_new)
    X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
    X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
    X_combined_new = scaler.transform(X_combined_new)

    # Make prediction
    prediction = model.predict(X_combined_new).round(1)[0]

    st.write(f'Predicted Audience Rating: {prediction}')



  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached tornado-6.4.2-cp38-abi3-win_amd64.whl.metadata (2.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/9.1 MB 7.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.0/9.1 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 6.6/9.1 MB 9.1 MB/s eta 0:00:01
   -------------------------------------- - 8.7/9.1 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 7.6 MB/s et


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip
2024-12-18 17:56:53.333 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 17:56:53.925 
  command:

    streamlit run C:\Users\91939\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-18 17:56:53.929 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 17:56:53.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 17:56:53.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 17:56:53.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 17:56

In [35]:
# prompt: streamlit run in local

import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# Load the trained model and vectorizer
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
scaler = joblib.load('scaler.pkl')

st.title('Movie Audience Rating Prediction')

# Input fields for movie features
genre = st.text_input('Genre', 'Comedy')
runtime = st.number_input('Runtime (minutes)', min_value=0, value=240)
tomatometer_rating = st.number_input('Tomatometer Rating', min_value=0, max_value=100, value=90)
tomatometer_count = st.number_input('Tomatometer Count', min_value=0, value=200)

if st.button('Predict'):
    # Create a DataFrame from the input features
    new_movie_data = pd.DataFrame({
        'genre': [genre],
        'runtime_in_minutes': [runtime],
        'tomatometer_rating': [tomatometer_rating],
        'tomatometer_count': [tomatometer_count]
    })

    # Preprocess the new movie data
    X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    X_tfidf_new = vectorizer.transform(X_text_new)
    X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
    X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
    X_combined_new = scaler.transform(X_combined_new)

    # Make prediction
    prediction = model.predict(X_combined_new).round(1)[0]

    st.write(f'Predicted Audience Rating: {prediction}')

2024-12-18 11:30:34.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:30:34.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [36]:
# prompt: 2024-12-18 11:30:34.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.

import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


# Load the trained model and vectorizer
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
scaler = joblib.load('scaler.pkl')

st.title('Movie Audience Rating Prediction')

# Input fields for movie features
genre = st.text_input('Genre', 'Comedy')
runtime = st.number_input('Runtime (minutes)', min_value=0, value=240)
tomatometer_rating = st.number_input('Tomatometer Rating', min_value=0, max_value=100, value=90)
tomatometer_count = st.number_input('Tomatometer Count', min_value=0, value=200)

if st.button('Predict'):
    # Create a DataFrame from the input features
    new_movie_data = pd.DataFrame({
        'genre': [genre],
        'runtime_in_minutes': [runtime],
        'tomatometer_rating': [tomatometer_rating],
        'tomatometer_count': [tomatometer_count]
    })

    # Preprocess the new movie data
    X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    X_tfidf_new = vectorizer.transform(X_text_new)
    X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
    X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
    X_combined_new = scaler.transform(X_combined_new)

    # Make prediction
    prediction = model.predict(X_combined_new).round(1)[0]

    st.write(f'Predicted Audience Rating: {prediction}')

2024-12-18 11:31:10.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 11:31:10.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [37]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=e8168ab20b437be381f8f1df48eb9a5566f44da6e5a71961bb34858b11d7f50b
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [38]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# Load the trained model and vectorizer
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
scaler = joblib.load('scaler.pkl')

st.title('Movie Audience Rating Prediction')

# Input fields for movie features
genre = st.text_input('Genre', 'Comedy')
runtime = st.number_input('Runtime (minutes)', min_value=0, value=240)
tomatometer_rating = st.number_input('Tomatometer Rating', min_value=0, max_value=100, value=90)
tomatometer_count = st.number_input('Tomatometer Count', min_value=0, value=200)

if st.button('Predict'):
    # Create a DataFrame from the input features
    new_movie_data = pd.DataFrame({
        'genre': [genre],
        'runtime_in_minutes': [runtime],
        'tomatometer_rating': [tomatometer_rating],
        'tomatometer_count': [tomatometer_count]
    })

    # Preprocess the new movie data
    X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    X_tfidf_new = vectorizer.transform(X_text_new)
    X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
    X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
    X_combined_new = scaler.transform(X_combined_new)

    # Make prediction
    prediction = model.predict(X_combined_new).round(1)[0]

    st.success(f'Predicted Audience Rating: {prediction}') # Changed to st.success for better formatting

Writing app.py


In [40]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.190.129.139:8501

  Stopping...
^C


In [ ]:
# prompt: sample prediction

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Load the data (assuming the CSV is in the same directory or provide the correct path)
data = pd.read_csv('/content/Rotten_Tomatoes_Movies3.csv')

# ... (rest of your data preprocessing code from the previous response)

# Sample prediction for a new movie
# Example new movie data (replace with your actual data)
new_movie_data = pd.DataFrame({
    'genre': ['Comedy'],  # Example genre
    'runtime_in_minutes': [120], # Example runtime
    'tomatometer_rating': [90], # Example tomatometer rating
    'tomatometer_count': [200] # Example tomatometer count
})

# Preprocess the new movie data in the same way as the training data
X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
X_tfidf_new = vectorizer.transform(X_text_new)  # Use the same vectorizer
X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
X_combined_new = st.transform(X_combined_new) # Use the same scaler

# Use the trained model to predict the audience rating
model = Ridge() # Use the best performing model identified previously
model.fit(X_train, y_train)
predicted_rating = model.predict(X_combined_new)

print(f"Predicted Audience Rating for the new movie: {predicted_rating[0]}")

In [ ]:
# prompt: create pipepline # Preprocess the new movie data in the same way as the training data
# X_text_new = new_movie_data[['genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
# X_tfidf_new = vectorizer.transform(X_text_new)  # Use the same vectorizer
# X_numerical_new = new_movie_data[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]
# X_combined_new = np.hstack((X_tfidf_new.toarray(), X_numerical_new))
# X_combined_new = st.transform(X_combined_new) # Use the same scaler

# ... (Your existing code)

# Create a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('scaler', StandardScaler()),
    ('model', Ridge()) # You can change the model here (e.g., XGBRegressor)
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df[['genre', 'runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']],
    y, test_size=0.2, random_state=42
)


# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("R2 Score:", r2)

# ... (Rest of your existing code for other models and new movie prediction)

# New movie prediction using the pipeline:
new_movie_data = pd.DataFrame({
    'genre': ['Comedy'],  # Example genre
    'runtime_in_minutes': [120], # Example runtime
    'tomatometer_rating': [90], # Example tomatometer rating
    'tomatometer_count': [200] # Example tomatometer count
})

predicted_rating = pipeline.predict(new_movie_data)
print(f"Predicted Audience Rating for the new movie (using pipeline): {predicted_rating[0]}")

In [ ]:
from sklearn.linear_model import ridge_regression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model = SVR(kernel='rbf')
# model = ridge_regression()
# st = StandardScaler()
# X_train = st.fit_transform(X_train)
# X_test = st.transform(X_test)
# model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")
# print(f"Accuracy Score: {accuracy_score(y_test, y_pred)}")

Mean Squared Error: 202.64118841102393
R2 Score: 0.5091540179240825


In [ ]:
y_pred

array([89.21468635, 86.12864879, 80.99968876, ..., 60.58608032,
       64.52625306, 75.05785399])

In [ ]:
# prompt: Preprocess text (TF-IDF) → Combine with numerical data → Train model (e.g., XGBoost, Ridge Regression).

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Assuming 'movie_info' is the text column and 'audience_rating' is the target
X_text = df['movie_info','movie_title']
y = df['audience_rating']

# Numerical features
X_numerical = df[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]


# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')  # Adjust max_features as needed
X_tfidf = vectorizer.fit_transform(X_text)

# Combine text and numerical features
X_combined = np.hstack((X_tfidf.toarray(), X_numerical))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train Ridge Regression model
model = Ridge(alpha=1.0)  # Adjust alpha as needed
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

KeyError: ('movie_info', 'movie_title')

In [ ]:
# prompt: i want Preprocess text (TF-IDF) data['movie_title', 'movie_info']

from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(df['movie_info'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with the original DataFrame
df_processed = pd.concat([df.drop('movie_info', axis=1), tfidf_df], axis=1)

In [ ]:
df_processed.head()

,movie_title,rating,genre,runtime_in_minutes,tomatometer_rating,tomatometer_count,audience_rating,000,10,100,...,your,youth,youthful,zach,zero,zoe,zombie,zombies,zone,øve
0,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",83.0,49.0,144.0,53.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Please Give,R,Comedy,90.0,86.0,140.0,64.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,R,"Comedy, Romance",118.0,68.0,22.0,53.0,0.000000,0.192010,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12 Angry Men (Twelve Angry Men),NR,"Classics, Drama",95.0,100.0,51.0,97.0,0.000000,0.065113,0.0,...,0.0,0.062515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"20,000 Leagues Under The Sea",G,"Action & Adventure, Drama, Kids & Family",127.0,89.0,27.0,74.0,0.133979,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def preprocess_features(df):
    """Prepares features for training."""
    df = df.fillna('Unknown')
    # Assuming 'movie_info' is the text column and 'audience_rating' is the target
    # X_text = df['movie_info']
    y = df['audience_rating']

    # Numerical features
    X_numerical = df[['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count']]

    # TF-IDF Vectorization
    # vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    # X_tfidf = vectorizer.fit_transform(X_text)

    # Combine text and numerical features
    # X = np.hstack((X_tfidf.toarray(), X_numerical))

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_numerical, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test
# # Train Ridge Regression model
# model = Ridge(alpha=1.0)  # Adjust alpha as needed
# model.fit(X_train, y_train)

In [ ]:
# Build and train the model
def train_model(X_train, y_train):
    """Trains a regression model using Linear Regression."""
    # Preprocessing for numerical and categorical features
    numeric_features = ['tomatometer_rating', 'tomatometer_count', 'runtime_in_minutes']
    # categorical_features = ['genre']

    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        # ('cat', categorical_transformer, categorical_features)
    ])

    # Define pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor)
        # ('regressor', LinearRegression())
    ])

    # # Train model
    model = pipeline.fit(X_train, y_train)
    return model
    # return pipeline

In [ ]:
import logging

In [ ]:
models = {
    # "Random Forest": RandomForestRegressor(),
                # "Decision Tree": DecisionTreeRegressor(),
                # "Gradient Boosting": GradientBoostingRegressor(),
                "Linear Regression": LinearRegression(),
                # "XGBRegressor": XGBRegressor(),
                # "CatBoosting Regressor": CatBoostRegressor(verbose=False),
                # "AdaBoost Regressor": AdaBoostRegressor(),
            }
params = {
    "Linear Regression": {
        'regressor__fit_intercept': [True, False],  # Updated parameter name
    },
}


# params = {
                  # "Decision Tree": {
#                     'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
#                     # 'splitter':['best','random'],
#                     # 'max_features':['sqrt','log2'],
#                 },
#                 "Random Forest": {
#                     # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],

#                     # 'max_features':['sqrt','log2',None],
#                     'n_estimators': [8, 16, 32, 64, 128, 256]
#                 },
#                 "Gradient Boosting": {
#                     # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
#                     'learning_rate': [.1, .01, .05, .001],
#                     'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
#                     # 'criterion':['squared_error', 'friedman_mse'],
#                     # 'max_features':['auto','sqrt','log2'],
#                     'n_estimators': [8, 16, 32, 64, 128, 256]
#                 },
                # "Linear Regression": {
                    # 'fit_intercept': [True, False],
                    # 'normalize': [True, False]
                # },
#                 "XGBRegressor": {
#                     'learning_rate': [.1, .01, .05, .001],
#                     'n_estimators': [8, 16, 32, 64, 128, 256]
#                 },
#                 "CatBoosting Regressor": {
#                     'depth': [6, 8, 10],
#                     'learning_rate': [0.01, 0.05, 0.1],
#                     'iterations': [30, 50, 100]
#                 },
#                 "AdaBoost Regressor": {
#                     'learning_rate': [.1, .01, 0.5, .001],
#                     # 'loss':['linear','square','exponential'],
#                     'n_estimators': [8, 16, 32, 64, 128, 256]
#                 }
# }

def evaluate_models(X_train, X_test, y_train, y_test, models=models,params=params):
    """Evaluates the model and prints metrics."""

          #  model_report: dict = evaluate_models(
          #      X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, models=models, param=params)

    # The following lines were indented but should be outside the function definition

    # Initialize an empty dictionary to store model performance
    model_report: dict = {}

    # Iterate through the models and parameters
    for model_name, model in models.items():
        # Perform hyperparameter tuning if parameters are provided
        if model_name in params and params[model_name]:
            grid_search = GridSearchCV(model, params[model_name], cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)  # Fit the model here
            best_model = grid_search.best_estimator_
            model_report[model_name] = -grid_search.best_score_
        else:
            # Fit the model without hyperparameter tuning
            model.fit(X_train, y_train)  # Fit the model here for cases with no hyperparameter tuning
            y_pred = model.predict(X_test)
            model_report[model_name] = mean_squared_error(y_test, y_pred)
  # Store the MSE as the score

    # Rest of your code to find the best model and evaluate it

    # Get the best model score and name
    best_model_score = min(model_report.values())  # Find the minimum MSE
    best_model_name = list(model_report.keys())[list(model_report.values()).index(best_model_score)]
    best_model = models[best_model_name]

    # Check if a good model was found
    if best_model_score > 0.6:  # Check if MSE is greater than 0.6 (indicating poor performance)
        print("No best model found")

    logging.info(f"Best found model on both training and testing dataset")

    # Evaluate the best model
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("Mean Squared Error:", mse)
    print("R2 Score:", r2)

    return model_report

In [ ]:
!pip install scikit-learn xgboost catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [ ]:
def main():
    # # Load data
    # file_path = 'Rotten_Tomatoes_Movies3.csv'  # Replace with the actual dataset path
    # print("Loading dataset...")
    # dataset = load_data(file_path)

    # Preprocess data
    print("Preparing features...")
    X_train, X_test, y_train, y_test = preprocess_features(df)

    # Split data
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    print("Training model...")
    pre_model =train_model(X_train, y_train)

    # Save the model
    # print("Saving model...")
    # joblib.dump(model, 'rotten_tomatoes_audience_rating_model.pkl')


    models = {
    "Random Forest": RandomForestRegressor(),
                "Decision Tree": DecisionTreeRegressor(),
                "Gradient Boosting": GradientBoostingRegressor(),
                "Linear Regression": LinearRegression(),
                "XGBRegressor": XGBRegressor(),
                "CatBoosting Regressor": CatBoostRegressor(verbose=False),
                "AdaBoost Regressor": AdaBoostRegressor(),
            }

    for i,j in models.items():
        model = models[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Model: {i}")
        print("Mean Squared Error:", mse)
        print("R2 Score:", r2)
        print("\n")

    # Evaluate model
    # print("Evaluating model...")
    # evaluate_models(X_train, X_test, y_train, y_test, models=models,params=params)
    # evaluate_model(model, X_test, y_test

    print("Project completed successfully!")

if __name__ == "__main__":
    main()

Preparing features...
Training model...
Model: Random Forest
Mean Squared Error: 259.7597157423953
R2 Score: 0.37079912639122226


Model: Decision Tree
Mean Squared Error: 429.3346092937512
R2 Score: -0.03995229000792455


Model: Gradient Boosting
Mean Squared Error: 213.45909750282152
R2 Score: 0.48295042499310803


Model: Linear Regression
Mean Squared Error: 220.07079639911706
R2 Score: 0.4669352907383688


Model: XGBRegressor
Mean Squared Error: 230.28218930348152
R2 Score: 0.4422008267441123


Model: CatBoosting Regressor
Mean Squared Error: 216.8548608523873
R2 Score: 0.4747250646441832


Model: AdaBoost Regressor
Mean Squared Error: 239.07255458393843
R2 Score: 0.4209084354354893


Project completed successfully!


In [ ]:
def main():
    # # Load data
    # file_path = 'Rotten_Tomatoes_Movies3.csv'  # Replace with the actual dataset path
    # print("Loading dataset...")
    # dataset = load_data(file_path)

    # Preprocess data
    print("Preparing features...")
    X_train, X_test, y_train, y_test = preprocess_features(df)

    # Split data
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    print("Training model...")
    # model =
    pipeline = train_model(X_train, y_train) # Assign the pipeline returned by train_model to a variable

    # Modify your models dictionary to use the pipeline
    models["Linear Regression"] = pipeline

    # Save the model
    # print("Saving model...")
    # joblib.dump(model, 'rotten_tomatoes_audience_rating_model.pkl')

    # Evaluate model
    print("Evaluating model...")
    evaluate_models(X_train, X_test, y_train, y_test, models=models,params=params)
    # evaluate_model(model, X_test, y_test

    print("Project completed successfully!")

if __name__ == "__main__":
    main()

Preparing features...
Training model...
Evaluating model...


ValueError: Invalid parameter 'regressor' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['tomatometer_rating',
                                                   'tomatometer_count',
                                                   'runtime_in_minutes'])]))]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
### Rotten Tomatoes Sentiment Analysis Project
# Offline Assignment for Zoho Corporation

# Import necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Load and preprocess the dataset
def load_data(file_path):
    """Loads Rotten Tomatoes dataset from a CSV file."""
    df = pd.read_csv(file_path)
    df.dropna(subset=['audience_rating'], inplace=True)
    return df

def preprocess_features(df):
    """Prepares features for training."""
    df = df.fillna('Unknown')
    X = df[['tomatometer_rating', 'tomatometer_count', 'genre', 'runtime_in_minutes']]
    y = df['audience_rating']
    return X, y

# Build and train the model
def train_model(X_train, y_train):
    """Trains a regression model using Linear Regression."""
    # Preprocessing for numerical and categorical features
    numeric_features = ['tomatometer_rating', 'tomatometer_count', 'runtime_in_minutes']
    categorical_features = ['genre']

    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

    # Define pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])

    # Train model
    model = pipeline.fit(X_train, y_train)
    return model

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    """Evaluates the model and prints metrics."""
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("Mean Squared Error:", mse)
    print("R2 Score:", r2)

# Main execution flow
def main():
    # Load data
    file_path = 'Rotten_Tomatoes_Movies3.csv'  # Replace with the actual dataset path
    print("Loading dataset...")
    dataset = load_data(file_path)

    # Preprocess data
    print("Preparing features...")
    X, y = preprocess_features(dataset)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    print("Training model...")
    model = train_model(X_train, y_train)

    # Save the model
    print("Saving model...")
    joblib.dump(model, 'rotten_tomatoes_audience_rating_model.pkl')

    # Evaluate model
    print("Evaluating model...")
    evaluate_model(model, X_test, y_test)

    print("Project completed successfully!")

if __name__ == "__main__":
    main()


In [ ]:
# Check if any column's data type is not 'object'
if any(data.dtypes != object):
  print(data.columns)


Index(['movie_title', 'movie_info', 'critics_consensus', 'rating', 'genre',
       'directors', 'writers', 'cast', 'in_theaters_date', 'on_streaming_date',
       'runtime_in_minutes', 'studio_name', 'tomatometer_status',
       'tomatometer_rating', 'tomatometer_count', 'audience_rating'],
      dtype='object')


In [ ]:
if any(data.dtypes == ):
  print(data.columns)

Index(['movie_title', 'movie_info', 'critics_consensus', 'rating', 'genre',
       'directors', 'writers', 'cast', 'in_theaters_date', 'on_streaming_date',
       'runtime_in_minutes', 'studio_name', 'tomatometer_status',
       'tomatometer_rating', 'tomatometer_count', 'audience_rating'],
      dtype='object')


AttributeError: 'numpy.ndarray' object has no attribute 'count'